## Two Constrained Optimisation
 **Author: Souvik Nath**
 <br>**Date: 7.19.2019**</br>

### Importing libraries

In [1]:
import pandas as pd
import os
import numpy as np
from IPython.display import display

In [4]:
os.getcwd()
os.chdir('C:\\Users\\souvik.nath\\Desktop')

### Reading the necessary files

In [5]:
tsh=pd.read_csv('TSH.csv')
cch=pd.read_csv('CCH.csv')
sch=pd.read_csv('SCH.csv')

In [6]:
tsh.columns = tsh.columns.str.lower()
cch.columns = cch.columns.str.lower()
sch.columns = sch.columns.str.lower()

In [7]:
display(tsh.head())
display(cch.head())
display(sch.head())

,str_nbr,sales
0,105,100250
1,106,1232890
2,107,172712
3,108,123133
4,109,724332


,dept,class,sales
0,21,1,23134
1,21,2,253524
2,21,3,335356
3,21,4,134535
4,21,5,332452


,str_nbr,dept,class,sales
0,105,21,1,1232
1,105,21,2,2234
2,105,21,3,5344
3,105,21,4,6543
4,105,21,5,2121


### Check for Equality in constraints
** <font color=red>NOTE: *If constraints don't match, then revisit the files and treat the differences*</font>**

In [8]:
def checkconstraint(x,y):
    if x.sum()==y.sum():
        print('Constraint: True')
    else:
        print('Constraint: Flase')

In [9]:
checkconstraint(tsh.iloc[:,-1],cch.iloc[:,-1])

Constraint: True


#### Concatenate department and class columns to create one composite key

In [10]:
def combinecolumn(x):
    if set(['dept','class']).issubset(x.columns):
        x['deptclass']=x['dept'].map(str)+x['class'].map(str)
        y=x[x.columns.difference(['dept','class'])]
        return y
    else:
        return False

### Declaring functions for applying absolute gross up methodology on SCH to CCH

In [11]:
def totaldeptsales(x):
    
    a=x.groupby(['deptclass'])['sales'].sum().reset_index()
    return a 

def deptsalesdiff(x,y):
    
    df1=totaldeptsales(x)
    df2=totaldeptsales(y)
    
    res = pd.DataFrame(df1['deptclass'])
    res['deptsalesdiff'] = df2.sales-df1.sales
    
    return res

In [12]:
def mutatecolumn(df, type):
    
    if type == 'dept':
        df['totaldeptsales'] = df['sales'].groupby(df['deptclass']).transform('sum')
    elif type == 'store':
        df['totalstrsales']=df['sales'].groupby(df['str_nbr']).transform('sum')
    
    return df


In [13]:
def calc_dept_pen(x):
    
    df = mutatecolumn(x,type='dept')
    df['dept_pen'] = df['sales']/df['totaldeptsales']
    
    return df

def mutatesalesdiff_dept(x,y):
    
    df1 = calc_dept_pen(x)
    df2 = deptsalesdiff(x,y)
    
    a=pd.merge(df1,df2,on='deptclass',how='left')
    a=pd.DataFrame(a)
    return a

### Declaring functions for applying absolute gross up methodology on SCH new to TSH

In [14]:
def totalstrsales(x):
    
    a=x[['str_nbr','sales']]
    b=pd.DataFrame(a.groupby(['str_nbr'])['sales'].sum().reset_index())
    
    res=pd.DataFrame(b)
    
    return res

def strsalesdiff(x,y):
    
    df1 = totalstrsales(x)
    df2 = y
    
    res = pd.DataFrame(df1['str_nbr'])
    res['strsalesdiff'] = df2.sales-df1.sales
    
    return res

def calc_str_pen(x):
    
    df = mutatecolumn(x, type='store')
    df['str_pen'] = df['sales']/df['totalstrsales']
    
    return df

In [15]:
def mutatesalesdiff_str(x,y):
    
    df1 = calc_str_pen(x)
    df2 = strsalesdiff(x,y)
    
    a=pd.merge(df1,df2,on='str_nbr',how='left')
    a=pd.DataFrame(a)
    return a   

### Function to calculate new sales using absolute gross up methodology

In [16]:
def newsales(x,y, type):
    
    if type=='dept':
        
        df = mutatesalesdiff_dept(x,y)
        df['newsales0']=(df['dept_pen']*df['deptsalesdiff'])+df['sales']
        df['newsales0']=df['newsales0']

        df_res = df[['str_nbr','deptclass','newsales0']]
        df_res.columns=['str_nbr', 'deptclass','sales']
        res=pd.DataFrame(df_res)
        
    elif type=='store':
        
        df = mutatesalesdiff_str(x,y)
        df['newsales0']=(df['str_pen']*df['strsalesdiff'])+df['sales']
        df['newsales0']=df['newsales0']

        df_res = df[['str_nbr','deptclass','newsales0']]
        df_res.columns=['str_nbr', 'deptclass','sales']
        res=pd.DataFrame(df_res )
    
    return res

In [17]:
sch=combinecolumn(sch)
cch=combinecolumn(cch)

In [18]:
tsh_diff=100
cch_diff=100

while any((tsh_diff > 0.01, cch_diff > 0.01)):
    sch_cch=newsales(sch,cch,type='dept')
    sch=sch_cch

    sch_tsh=newsales(sch,tsh,type='store')
    sch=sch_tsh

    cch_diff=max(abs(deptsalesdiff(sch,cch)['deptsalesdiff']))
    tsh_diff=max((abs(strsalesdiff(sch,tsh)['strsalesdiff'])))
    
    print('cch_diff: %d; tsh_diff: %d' %(cch_diff, tsh_diff))

cch_diff: 1411245; tsh_diff: 0
cch_diff: 326385; tsh_diff: 0
cch_diff: 82206; tsh_diff: 0
cch_diff: 21544; tsh_diff: 0
cch_diff: 5740; tsh_diff: 0
cch_diff: 1541; tsh_diff: 0
cch_diff: 415; tsh_diff: 0
cch_diff: 112; tsh_diff: 0
cch_diff: 30; tsh_diff: 0
cch_diff: 8; tsh_diff: 0
cch_diff: 2; tsh_diff: 0
cch_diff: 0; tsh_diff: 0
cch_diff: 0; tsh_diff: 0
cch_diff: 0; tsh_diff: 0
cch_diff: 0; tsh_diff: 0
cch_diff: 0; tsh_diff: 0
